# GWW Single algorithm using EE

First an example of the algorithm using earthengine is generated for comparison with the openEO implementation.

In [ ]:
import ee
import geemap

## Obtain basins dataset

This algorithm uses a dataset that collects a few open source datasets of basins as polygons.
Right now, this is saved as an earthengine asset.
We also get the JRC water occurence dataset

In [ ]:
from pathlib import Path
from typing import List
from utils import Reservoir

out_dir = Path.cwd() / "output"
reservoir_dir: Path = out_dir / "reservoirs"

reservoirs: List[Reservoir] = Reservoir.from_gcp(reservoir_dir)

In [ ]:
import json
import shapely

geojson_str = json.dumps(shapely.geometry.mapping(reservoirs[1].geometry))

In [ ]:
import geojson
# Selected an area in https://code.earthengine.google.com/7599feb42d93ad763852d6b78fc98930
# geojson_str = "{\"type\":\"Polygon\",\"coordinates\":[[[16.00079546305277,49.19952794145144],[16.00079546305277,49.12049851321871],[16.125764945474646,49.12049851321871],[16.125764945474646,49.19952794145144]]],\"geodesic\":false,\"evenOdd\":true}"
bbox: geojson.Polygon = geojson.loads(geojson_str)
Map: geemap.Map = geemap.Map()

sac = ee.ServiceAccountCredentials("dagster-workloads@global-water-watch.iam.gserviceaccount.com", "/home/jovyan/work/notebooks/pk.json")
ee.Initialize(sac)

In [ ]:
water_occurrence = ee.Image("JRC/GSW1_3/GlobalSurfaceWater") \
  .select('occurrence') \
  .unmask(0) \
  .resample('bicubic') \
  .divide(100)
  
water_occurrence = water_occurrence.mask(water_occurrence)
waterbodies = ee.FeatureCollection("projects/global-water-watch/assets/reservoirs-v1-0")

count = waterbodies.size().getInfo()
count

## Obtain basins within area

Ideally, a service containing the GWW algorithm would provide data and metadata for a selected bounding box and timeframe. In this notebook, we focus on a single basin. Here we select an area of interest and get all basins within this area.

In [ ]:
bbox: ee.Geometry = ee.Geometry(bbox)
waterbody: ee.Feature = ee.Feature(waterbodies.filterBounds(bbox).first())  # Only 1 basin in bbox
Map.centerObject(bbox)
Map.addLayer(waterbody, {"opacity": 0.8}, "waterbody_area")
Map.addLayer(water_occurrence, {}, "woc")
Map

In [ ]:
waterbody.get("fid").getInfo()

## Use single basin analysis from eepackages implementation

We use the eepackages implementation to obtain the waterbody area and other statistics for a certain timeframe within the waterbody (internally it buffers the waterbody)

In [ ]:
from eepackages.applications.waterbody_area import computeSurfaceWaterArea

start_filter = "2021-05-01"
start = "2021-09-01"
stop = "2022-05-01"
scale = waterbody.geometry().area().sqrt().divide(200).max(10).getInfo()
# missions = ["S2"]
missions = ["L4", "L5", "L7", "L8", "S2"]

water_area: ee.ImageCollection = computeSurfaceWaterArea(waterbody, start_filter, start, stop, scale, water_occurrence, missions)
# Filter out poor images
water_area = water_area.filter(
        ee.Filter.And(
          ee.Filter.neq('p', 101),
          ee.Filter.gt('ndwi_threshold', -0.15),
          ee.Filter.lt('ndwi_threshold', 0.5),
          ee.Filter.lt('filled_fraction', 0.6)
        )
      ).sort("system:time_start")

properties = [
    "MISSION",
    "ndwi_threshold",
    "quality_score",
    "area_filled",
    "filled_fraction",
    "p",
    "system:time_start",
    "area",
]
properties_new = [
    "mission",
    "ndwi_threshold",
    "quality_score",
    "water_area_filled",
    "water_area_filled_fraction",
    "water_area_p",
    "water_area_time",
    "water_area_value",
]

water_area = (
    ee.FeatureCollection(water_area)
    .select(properties, properties_new, False)
    .set("scale", scale)
)

# Check amount of images remaining
print(f"number of images in FC: {water_area.size().getInfo()}")
# water_area_ex: ee.Image = water_area.filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10).first()
# Map.addLayer(water_area_ex, {"opacity": 0.8, "bands": ["water"]}, "water_area_ex")
# Map.addLayer(water_area_ex, {"opacity": 0.8, "bands": ["water_fill"]}, "water_fill")
# Map.addLayer(water_area_ex, {"opacity": 0.8, "bands": ["water_edge"]}, "water_edge")
Map

## Obtain statistics of water area over time

Water area is used as a proxy of available capacity of the basin. We take the available water area over time.

In [ ]:
# filled_area is water area after correcting for false positives
water_statistics = water_area.getInfo()

In [ ]:
water_statistics

In [ ]:
from datetime import datetime
import holoviews as hv
from holoviews import opts

hv.extension("bokeh")

xy = zip(*map(lambda prop: (datetime.fromtimestamp(prop["system:time_start"] / 10e2), prop["water_filled_area"]),
         map(lambda feat: feat["properties"], water_statistics["features"])
))
x = list(next(xy))
y = list(next(xy))

hv.Dimension.type_formatters[datetime] = '%Y-%m-%d'
time = hv.Dimension("time", label="time" )
area = hv.Dimension("area", label="area", unit="m2")

hv.Curve((x, y), time, area).opts(opts.Curve(height=200, width=900, line_width=1.50, color='black', tools=['hover']))

In [ ]:
len(water_statistics["features"])